# Convert from md format to instructor version and public version

In [1]:
import yaml
from pathlib import Path
import sys
import numpy as np

In [2]:
# deal with multi-line strings in YAML Dump
## Code copied from here: https://stackoverflow.com/a/33300001/2217577

def str_presenter(dumper, data):
    if len(data.splitlines()) > 1:  # check for multiline string
        return dumper.represent_scalar('tag:yaml.org,2002:str', data, style='|')
    return dumper.represent_scalar('tag:yaml.org,2002:str', data)

yaml.add_representer(str, str_presenter)

## Declare functions

In [3]:
def rounded(num, digits_after_decimal = 2):
    
    """
    Rounding as expected by normal sensible people.
    
    This needs to be heavily tested!!
    
    WARNING: This does not do sig figs yet!
    
    """

    # Solution copied from: https://stackoverflow.com/a/53329223

    from decimal import Decimal, getcontext, ROUND_HALF_UP

    round_context = getcontext()
    round_context.rounding = ROUND_HALF_UP
        
    tmp = Decimal(num).quantize(Decimal('1.'+'0'*digits_after_decimal))
    
    return str(tmp)

## Turn the interactive code (at the bottom of this notebook) into a function

In [4]:
import yaml
from pathlib import Path
import sys
import numpy as np

def process_source_files(source_file, instructor=True, prairielearn=False, public=True, qti=False, webwork=False):
    """
    Inputs:
    
    - path to source file
    - output types
    
    Outputs:
    
    - processed files to each of the output types
    """
    
    header = yaml.safe_load(Path(source_file).read_text().split("---\n# {{")[0]) # This code only takes the YAML header portion
    
    # populate the data dictionary after running generate function
    data={'params':{},
          'vars': {}}

    ## TODO: Is there a better way to do this?
    exec(header['server'])

    # This is a Completely incomrephensible dictionary comprehension
    # All this does is take every number in the data['params'] dictionary, and turns it into a sensibly rounded number, 
    # It also leaves strings as-is (like correct-answer)

    data['params'] = {key : rounded(data['params'][key], digits_after_decimal=data["vars"]["digits_after_decimal"]) if type(data['params'][key]) != str else data['params'][key] for key in data['params']}

    # Prepare file for export
    part1 = '---\n'
    part3 = '---'
    part4 = Path(source_file).read_text().split("---")[2]
    
    if instructor:
        
        # Load the header of the file again
        instructor_header = yaml.safe_load(Path(source_file).read_text().split("---\n# {{")[0]) # This code only takes the YAML header portion
        
        # Update the YAML header to add substitutions 
        instructor_header.update({'substitutions':data})
        
        # Update the YAML header to add substitutions, unsort it, and process for file
        instructor_header = yaml.dump(instructor_header,sort_keys=False,default_flow_style=False)

        part2 = instructor_header

        # Write the YAML to a file
        path = Path(source_file.replace('source','output/instructor'))
        path.parent.mkdir(parents=True, exist_ok=True)
        path.write_text(part1 + part2 + part3 + part4)
        
    if public:
        
        sanitized_header = yaml.safe_load(Path(source_file).read_text().split("---\n# {{")[0]) # This code only takes the YAML header portion

        # Remove python solution from the public view
        sanitized_header.pop('server',None)

        # Remove the correct answer from the dictionary
        data["params"]["correct_answer"] = 'nan'

        # Update the YAML header to add substitutions 
        sanitized_header.update({'substitutions':data})

        # Update the YAML header to add substitutions, unsort it, and process for file
        sanitized_header = yaml.dump(sanitized_header,sort_keys=False,default_flow_style=False)

        part2 = sanitized_header

        # Write the YAML to a file
        path = Path(source_file.replace('source','output/public'))
        path.parent.mkdir(parents=True, exist_ok=True)
        path.write_text(part1 + part2 + part3 + part4)
    if webwork:
        raise NotImplementedError
        
    if prairielearn:
        raise NotImplementedError
    

In [5]:
process_source_files(source_file = 'source/q1_multiple-choice/q1_multiple-choice.md')
process_source_files(source_file = 'source/q2_number-input/q2_number-input.md')


ComposerError: expected a single document in the stream
  in "<unicode string>", line 2, column 1:
    title: Distance travelled
    ^
but found another document
  in "<unicode string>", line 60, column 1:
    ---
    ^

In [ ]:
process_source_files(source_file = 'source/half-life/halflife.md')

## Legacy stuff below this

### Read YAML file

### Export to Instructor version of new YAML file

### Export to Public version of new YAML file